In [11]:
json_file = r"C:\Users\psh10\Documents\pythonEx\project\pythonProject\book-recommendation\data\scraping\llm_output[0_ 1486].json"
with open(json_file, 'r', encoding='utf-8') as file:
            data = json.load(file)

# NULL 값 찾기
null_entries = {key: value for key, value in data.items() if value is None}
print(f"NULL 값이 포함된 항목: {null_entries}")

NULL 값이 포함된 항목: {}


In [25]:
import re
import psycopg2
import json

json_file = r"C:\Users\psh10\Documents\pythonEx\project\pythonProject\book-recommendation\data\scraping\llm_output[0_ 1486].json"
with open(json_file, 'r', encoding='utf-8') as file:
            data = json.load(file)

# NULL 값 찾기
null_entries = {key: value for key, value in data.items() if value is None}
print(f"NULL 값이 포함된 항목: {null_entries}")

# 데이터베이스 연결 설정
conn = psycopg2.connect(
    dbname="book_recommend",
    user="sesac",
    password="1234",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# 새로운 테이블 생성 (isbn, message, hashtags 저장)
cursor.execute("""
    CREATE TABLE IF NOT EXISTS new_books (
        isbn VARCHAR(30) PRIMARY KEY,
        message TEXT,
        hashtags TEXT
    );
""")
conn.commit()

try:
    # 데이터 삽입
    for book in data.values():
        if book is None:
            continue  # NULL 값 무시
        
        isbn = str(book.get("isbn13", ""))
        message = book.get("message", "")
        hashtags = " ".join([tag.strip() for tag in book.get("hashtags", "").split() if tag.strip()])
        
        # new_books 테이블에 데이터 삽입
        cursor.execute("INSERT INTO new_books (isbn, message, hashtags) VALUES (%s, %s, %s) ON CONFLICT (isbn) DO NOTHING", (isbn, message, hashtags))
    
    # 변경 사항 저장
    conn.commit()
except Exception as e:
    print(f"오류 발생: {e}")
    conn.rollback()
finally:
    # 연결 종료
    cursor.close()
    conn.close()

NULL 값이 포함된 항목: {}


In [14]:
conn.rollback()

InterfaceError: connection already closed